### Let's read a text file from HDFS to an RDD and cache it in memory

In [1]:
tweets = sc.textFile("congress-sample-10k.json.gz").cache() # cache the file content in memory.

NameError: name 'sc' is not defined

### Show a sample

In [2]:
tweets.take(1)

[u'{"extended_tweet":{"entities":{"urls":[],"hashtags":[],"user_mentions":[{"indices":[0,8],"screen_name":"jillwow","id_str":"464222379","name":"jillie","id":464222379},{"indices":[9,23],"screen_name":"RepAdamSchiff","id_str":"29501253","name":"Adam Schiff","id":29501253}],"symbols":[]},"full_text":"@jillwow @RepAdamSchiff That is what is so disturbing &amp; ominous \U0001f633 Also, we must remember our world history - didn\\u2019t Hiler only have 30% or so support from the German ppl before he began his persecution of the Jewish ppl? \U0001f615","display_text_range":[24,226]},"in_reply_to_status_id_str":"1047522510839975936","in_reply_to_status_id":1047522510839975936,"created_at":"Wed Oct 03 17:30:03 +0000 2018","in_reply_to_user_id_str":"464222379","source":"<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone<\\/a>","retweet_count":0,"retweeted":false,"geo":null,"filter_level":"low","in_reply_to_screen_name":"jillwow","is_quote_status":false,"id_st

### Let's force all data to memory

In [3]:
tweets.count() # count() is an action (eagerly executed)

10000

### Let's do MapReduce in Spark (word count)

# import json # we need this to parse the JSON tweet objects (tweets are seperated by new lines)
words = tweets.flatMap(lambda line: json.loads(line)['text'].split(" ")) # split each line by space
words.take(10)

In [5]:
# mapper function: emit (word, 1)
word_tuples = words.map(lambda word: (word,1))
word_tuples.take(10)

[(u'@jillwow', 1),
 (u'@RepAdamSchiff', 1),
 (u'That', 1),
 (u'is', 1),
 (u'what', 1),
 (u'is', 1),
 (u'so', 1),
 (u'disturbing', 1),
 (u'&amp;', 1),
 (u'ominous', 1)]

In [6]:
# reducer function: sum all counts
word_counts = word_tuples.reduceByKey(lambda count1, count2 : count1+count2)
word_counts.take(10)

[(u'', 964),
 (u'considered.', 1),
 (u'https://t.co/KWy5MfolA7', 1),
 (u'EXPLAIN', 2),
 (u'protested,', 3),
 (u'@FrancaBrilliant:', 1),
 (u'https://t.co/NZPp2ILXEV', 1),
 (u'@ThomasB00001', 1),
 (u'hanging', 2),
 (u'government?!Investigate', 1)]

In [7]:
word_counts.sortBy(lambda x: -x[1]).take(10) # sort by value

[(u'RT', 6714),
 (u'the', 5256),
 (u'to', 4126),
 (u'a', 2817),
 (u'of', 2379),
 (u'and', 2225),
 (u'is', 2074),
 (u'in', 1693),
 (u'for', 1489),
 (u'you', 1411)]

# DataFrames

In [8]:
tweets = spark.read.json("congress-sample-10k.json.gz").cache()

In [9]:
tweets.printSchema()

root
 |-- contributors: string (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |    |-- type: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: long (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |    |    |-- embeddable: boolean (nullable = true)
 |    |    |    |    |-- monetizable: bo

In [10]:
# show a sample ignoring retweets
tweets.filter("retweeted_status is null")\
      .select("user.screen_name", "text")\
      .sample(0.001).show(truncate=False)

+-----------+----+
|screen_name|text|
+-----------+----+
+-----------+----+



# Exploratory Data Analysis

### How many unique users, tweets, retweets, quotes, and quoted?

In [11]:
# unique users
tweets.select("user.id").distinct().count()

9735

In [12]:
# original tweets
tweets.filter("retweeted_status is null").count()

3327

In [13]:
# retweets
tweets.filter("retweeted_status is not null").count()

6673

In [14]:
# quotes
tweets.filter("quoted_status is not null").count()

2722

In [15]:
# quoted
tweets.filter("quoted_status is not null").select("quoted_status.id").distinct().count()

2200

### What are the top hashtags used?

In [16]:
# We will utilize some of the Spark dataframe builtin functions
import pyspark.sql.functions as F

In [17]:
# hashtags field is an arry, we want to *explode* it to extract the elements. 
# Also we only care about the text of the hashtag
tweets.select(F.explode("entities.hashtags.text").alias("hashtag"))\
      .groupby("hashtag").count().sort(F.desc("count")).show()

+-----------------+-----+
|          hashtag|count|
+-----------------+-----+
|        Venezuela|  102|
|    TrumpShutdown|   42|
|     MaduroRegime|   29|
|           Maduro|   20|
|             MAGA|   20|
|      NancyPelosi|   19|
|    MuellerReport|   17|
|     BuildTheWall|   14|
|      TrumpResign|   14|
|        Kavanaugh|   14|
|     ForThePeople|   14|
|     GreenNewDeal|   13|
| MyHouseMyAmerica|   12|
|        transport|   12|
|          Florida|   10|
|             Cuba|   10|
|        Democrats|   10|
|    BrowardCounty|    9|
|             EEUU|    8|
|MaduroCrimeFamily|    8|
+-----------------+-----+
only showing top 20 rows



### Who are top followed users?

In [18]:
tweets.select("user.screen_name", "user.followers_count")\
      .groupby("screen_name").agg(F.max("followers_count").alias("followers_count"))\
      .sort(F.desc("followers_count")).show()

+---------------+---------------+
|    screen_name|followers_count|
+---------------+---------------+
|        FoxNews|       18104837|
|      ANCALERTS|        4690989|
|     CoryBooker|        4077716|
|     marcorubio|        3614212|
|  rapplerdotcom|        3054285|
|  SenGillibrand|        1290510|
|   SethAbramson|         623486|
|  SenatorDurbin|         577839|
|   SenDuckworth|         517255|
|        easyJet|         459166|
|       tribelaw|         448458|
|    HenshawKate|         444390|
|     MarkWarner|         441319|
|        AppSame|         354567|
|SenatorCantwell|         225930|
|      LeedsNews|         223915|
|   BelenMarrero|         223378|
|           WFTV|         203174|
| SouthernRailUK|         189159|
|     Daily_Star|         181050|
+---------------+---------------+
only showing top 20 rows



### Who gained the most followers?

In [19]:
# remeber this is just in the 10k sample.
tweets.select("user.screen_name", "user.followers_count")\
      .groupby("screen_name").agg((F.max("followers_count")-F.min("followers_count")).alias("followers_diff"))\
      .sort(F.desc("followers_diff")).show()

+---------------+--------------+
|    screen_name|followers_diff|
+---------------+--------------+
|ArthurSamuelHu1|         15469|
|HernandoDeSot11|          9505|
|       Jali_Cat|          6882|
|ChristusPatriot|          6102|
|   myserenity69|          5646|
|       BOURRELL|          4347|
|    MagaGoldHat|          3953|
|ccbandit4resist|          3756|
|  sheaffer_jean|          3204|
|        Brenro4|          3102|
|   TNHORSEFARMS|          2898|
|   FrakerMonica|          2609|
|  openletterbot|          2489|
|       enja1949|          1995|
| TheConsulyetti|          1921|
|KarmaSQuirrel17|          1857|
|     Clover1292|          1702|
|   DanielDastti|          1682|
|      skibbers2|          1627|
|   thomaskaine5|          1501|
+---------------+--------------+
only showing top 20 rows



### Who is the most mentioned?

In [20]:
tweets.select(F.col("user.screen_name").alias("user"), 
              F.explode("entities.user_mentions.screen_name").alias("mention"))\
      .groupby("mention").count().sort(F.desc("count")).show()

+---------------+-----+
|        mention|count|
+---------------+-----+
|     SenSchumer|  776|
|realDonaldTrump|  751|
|  RepAdamSchiff|  739|
|     marcorubio|  695|
|  SpeakerPelosi|  505|
|    NancyPelosi|  368|
|       RandPaul|  272|
|  ChrisMurphyCT|  223|
|   RepMattGaetz|  220|
|  SenGillibrand|  220|
|     CoryBooker|  214|
|  ChuckGrassley|  206|
|      JeffFlake|  202|
|   SteveScalise|  179|
|   amyklobuchar|  176|
|      GOPLeader|  164|
| RepJerryNadler|  162|
|          POTUS|  159|
|SenKamalaHarris|  152|
| SenJeffMerkley|  141|
+---------------+-----+
only showing top 20 rows



### Find mentioned users who never tweeted

### View lineage

In [21]:
# This is helpful ot know how much memory your dataframe is using and 
# the transfomration performed to get to the current state.
print tweets.rdd.toDebugString()

(1) MapPartitionsRDD[122] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[121] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[120] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  MapPartitionsRDD[119] at javaToPython at NativeMethodAccessorImpl.java:0 []
 |  FileScan json [contributors#6,coordinates#7,created_at#8,display_text_range#9,entities#10,extended_entities#11,extended_tweet#12,favorite_count#13L,favorited#14,filter_level#15,geo#16,id#17L,id_str#18,in_reply_to_screen_name#19,in_reply_to_status_id#20L,in_reply_to_status_id_str#21,in_reply_to_user_id#22L,in_reply_to_user_id_str#23,is_quote_status#24,lang#25,place#26,possibly_sensitive#27,quote_count#28L,quoted_status#29,... 13 more fields] Batched: false, Format: JSON, Location: InMemoryFileIndex[hdfs://nn:8020/user/dreamer/congress-sample-10k.json.gz], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<contributors:string,coordinates:struct<coordinates:arra

### Saving a dataframe for later analysis

In [22]:
# parquet is a columnar file format with good performance on queries like the above
tweets.write.parquet("/tmp/mytweets") 

In [23]:
### read parquet
mytweets = spark.read.parquet("/tmp/mytweets").cache()